In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Avant de passer aux pré-traitements importons les librairies nécessaires

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import seaborn as sns

### Récupérons le dataset

In [ ]:
file_path = "../input/students-performance-in-exams/StudentsPerformance.csv"

In [ ]:
exam_perf = pd.read_csv(file_path)

In [ ]:
exam_perf.head()

In [ ]:
exam_perf.shape

In [ ]:
# changement des noms des colonnes (certaines)
exam_perf.rename(columns = {"race/ethnicity": "race_ethnicity",
                            "parental level of education": "parental_lev_education",
                            "test preparation course": "test_preparation",
                            "math score": "math_score",
                            "reading score": "reading_score",
                            "writing score": "writing_score"}, inplace = True)

In [ ]:
exam_perf.describe()

### Vérifions les types des données et transformons les types des données categorielles en 'category'

In [ ]:
exam_perf.dtypes

In [ ]:
categorial_features = ["gender", "race_ethnicity", "parental_lev_education", 
                       "lunch", "test_preparation"]
exam_perf[categorial_features] = exam_perf[categorial_features].astype("category")

### Déterminons si les colonnes math_score, reading_score et writing_score sont bien corrélées à travers quatre tests

In [ ]:
# On vérifie s'il y a des données a partir du tracage d'un boxplot
# La vérification se fera par genre
sns.set_theme(rc = {"axes.spines.right": False, "axes.spines.top": False})

In [ ]:
# Voyons les relations entre les targets avec seaborn
sns.pairplot(data = exam_perf, hue="gender")
plt.title("relational plot between math_score, writing_score and reading_score")
plt.savefig("pairplot_score_features.png")

Nous voyons que les trois variables suivent des distributions normales et que les scatterplots entre les variables nous montrent qu'il existe une corrélation entre les différentes variables

In [ ]:
# Tracons les lmplots pour voir la linearite des relations entre
# les variables
variables = ["math_score", "reading_score", "writing_score"]
fig, ax = plt.subplots(3, 1, figsize = (15, 8))
ax = ax.flat
fig.tight_layout(pad = 4.5)
sns.regplot(x = variables[0], y = variables[1], data = exam_perf, ax = ax[0])
ax[0].set_title("Linear model of reading_score/mah_score", fontsize = 16)
sns.regplot(x = variables[0], y = variables[2], data = exam_perf, ax = ax[1])
ax[1].set_title("Linear model of writing_score/mah_score", fontsize = 16)
sns.regplot(x = variables[2], y = variables[1], data = exam_perf, ax = ax[2])
ax[2].set_title("Linear model of reading_score/writing_score", fontsize = 16)
plt.savefig("linear_models.png")

In [ ]:
# Tracons le heatmap entre nos trois variables
sns.heatmap(exam_perf[variables].corr(), annot = True)
plt.title("Correlation between math_score, reading_score and writing_score")
plt.savefig("Heatmap_score_features")


On note une forte corrélation entre les variables

### Vérifions si le dataset contient des données aberrantes ou des valeurs manquantes
 

In [ ]:
exam_perf.isnull().sum()

Pas de traitement à faire pour des données manquantes

### Déterminons les moyennes obtenues dans une nouvelle colonne

In [ ]:
exam_perf['average'] = exam_perf.mean(axis = 1)

### Nous allons concevoir une fonction qui va classer les moyennes par pas de 20 :
- classe A : $average \in ]80, 100]$
- classe B : $average \in ]60, 80]$
- classe C : $average \in ]40, 60]$
- classe D : $average \in ]20, 40]$
- classe E : $average \in ]0, 20]$


A étant la classe la plus forte et E la classe la plus faible

In [ ]:
def make_classes(average):
  """
  Cette fonction va permettre déterminer la classe de chaque valeur de la colonne average
  """
  classes = ['A', 'B', 'C', 'D', 'E']
  i = 0
  for value in range(100, 9, -20):
      if (average <= value) and\
         (average > value-20):
         classe = classes[i]
      i += 1
  if average == 0:
    return 'E'
  return classe

In [ ]:
exam_perf['classes'] = exam_perf['average'].map(make_classes)

In [ ]:
exam_perf.head()

### Déterminons s'il y a des données aberrantes 

In [ ]:
# Verifions cela en prenant pour attribut hue de notre fonction boxplot 
# les differentes colonnes categorielles de notre dataset
fig, ax = plt.subplots(5, 1, figsize = (16, 14))
ax = ax.flat
for i, column in enumerate(categorial_features):
  fig.tight_layout()
  sns.boxplot(x = column, y = "average", data = exam_perf, ax = ax[i])
  ax[i].set_title("Boxplot for {}".format(column))
plt.savefig("boxplots.png")

## Essayons de réduire le nombre de données aberrantes

In [ ]:
descriptions = exam_perf['average'].describe()
descriptions = descriptions['25%':'75%']
interquartil = descriptions['75%'] - descriptions['25%']
limitation = descriptions['25%'] - 1.5*interquartil
exam_perf.drop(index = exam_perf[exam_perf['average']<= limitation].index.values, inplace=True)

Notre dataset est plus propre à présent

### Revérifions s'il y a des données aberrantes avec le boxplot

In [ ]:
# Verifions cela en prenant pour attribut hue de notre fonction boxplot 
# les différentes colonnes categorielles de notre dataset
fig, ax = plt.subplots(5, 1, figsize = (16, 14))
ax = ax.flat
for i, column in enumerate(categorial_features):
  fig.tight_layout()
  sns.boxplot(x = column, y = "average", data = exam_perf, ax = ax[i])
  ax[i].set_title("Boxplot for {}".format(column))
plt.savefig("boxplots2.png")

#### Parfait !!

In [ ]:
exam_perf.shape

Donc on a éliminé 7 lignes qui contenaient des données aberrantes

### On passe à l'entrainement de notre modèle


#### Transformons le type de notre variable classes en 'category'

In [ ]:
exam_perf['classes'] = exam_perf['classes'].astype('category')
exam_perf.dtypes

#### nous devons copier le dataset original et encoder les colonnes categorielles


In [ ]:
categorial_features.append('classes')

In [ ]:
exam_perf_prepared = exam_perf.copy()
for column in categorial_features:
  exam_perf_prepared[column] = exam_perf_prepared[column].cat.codes

#### Commencons par le math_score target

In [ ]:
target = "classes"
categorial_features.pop()
features = categorial_features

In [ ]:
features

In [ ]:
# Separons les donnees d'entrainement des donnees de test
X_train, X_test, y_train, y_test = train_test_split(exam_perf_prepared[features], exam_perf_prepared[target])

In [ ]:
neighbors_model = KNeighborsClassifier(n_neighbors=1)
neighbors_model.fit(X_train, y_train)
predictions = neighbors_model.predict(X_test)

In [ ]:
print("Score {}".format(neighbors_model.score(X_test, y_test)))

###Verifions le nombre de voisins pour notre model nous donnant le score le plus élevé

In [ ]:
def get_score(X_train, X_test, y_train, y_test):
  scores = np.array([])
  # best_number_of_neighbors = []
  for i in range(1, 11):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train, y_train)
    scores = np.concatenate((scores, [model.score(X_test, y_test), i]), axis = 0)
  scores = scores.reshape(10, 2)
  print("La meilleure prediction est de {}".format(scores[scores[:, 0] == np.max(scores[:, 0])]))

In [ ]:
get_score(X_train, X_test, y_train, y_test)

Donc 9 est le meilleur nombre de voisins pour notre modèle

### Réentrainons le modèle avec 9 pour n_neighbors cette fois ci

In [ ]:
def train(X_train, X_test, y_train, n):
  neighbors_model = KNeighborsClassifier(n_neighbors=n)
  neighbors_model.fit(X_train, y_train)
  predictions = neighbors_model.predict(X_test)
  print("Score {}".format(round(neighbors_model.score(X_test, y_test)*100, 2)))
  return neighbors_model

In [ ]:
train(X_train, X_test, y_train, 9)

### A present indrosuisons le math_score dans les features pour voir si la prédiction sera plus précise

In [ ]:
features.append('math_score')

In [ ]:
# Séparons les données d'entrainement des données de test
X_train, X_test, y_train, y_test = train_test_split(exam_perf_prepared[features], exam_perf_prepared[target])

In [ ]:
print("En rajoutant le score feature math_score nous avons :")
train(X_train, X_test, y_train, 9)

### Essayons avec les deux autres features reading_score et writing_score

In [ ]:
for score_feature in ['writing_score', 'reading_score']:
  features.pop()
  features.append(score_feature)
  # Separons les donnees d'entrainement des donnees de test
  X_train, X_test, y_train, y_test = train_test_split(exam_perf_prepared[features], exam_perf_prepared[target])
  print("En rajoutant le score feature {} nous obtenons".format(score_feature))
  model = train(X_train, X_test, y_train, 9)
  print("---------------------")

Donc uniquement avec le reading_score nous obtenons une meilleure prédiction qu'avec les deux autres

### Cross validation avec le feature reading_score

In [ ]:
# définissons les paramètres à tester
params = {
    "n_neighbors": np.arange(1, 20),
    "metric": ["manhattan", "euclidean", "minkowski"]
}

# cross validation et entraînement 
grid = GridSearchCV(KNeighborsClassifier(), params, cv = 5, scoring = "accuracy")
grid.fit(X_train, y_train)

In [ ]:
# meilleurs paramètres 
grid.best_params_

### Le meilleur score obtenu 

In [ ]:
model = grid.best_estimator_
print("Le meilleur score obtenu est de : {}".format(model.score(X_test, y_test)))

### Concevons une fonction qui nous permet de prédire le niveau auquel appartient notre candidat 

In [ ]:
def predire(gender = 0, race_ethnicity = 1, parental_lev_education = 2, lunch = 0, test_prep = 1, reading_score = 81):
  x = np.array([gender, race_ethnicity, parental_lev_education, lunch, test_prep, reading_score])
  x = x.reshape(1, x.shape[0])
  print("model prediction {}".format(model.predict(x)))
  print("model prediction probabilities {}".format(model.predict_proba(x)))

In [ ]:
predire()

Donc notre modèle a prédit que je suis de la classe B 

### Matrice de confusion

In [ ]:
print("Matrice de confusion \n{}".format(confusion_matrix(y_test, model.predict(X_test))))

### exportation des résultats 

In [ ]:
predictions = pd.DataFrame({"Id": X_test.index,
                            "predictions": model.predict(X_test)})
predictions.to_csv("students_performances_predict.csv")